In [ ]:
!rm -Rf sample_data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Data

In [ ]:
! pip -q install optuna
! pip -q install tensorflow-addons

     |████████████████████████████████| 302 kB 5.3 MB/s 
     |████████████████████████████████| 164 kB 44.5 MB/s 
     |████████████████████████████████| 80 kB 9.0 MB/s 
     |████████████████████████████████| 75 kB 4.4 MB/s 
     |████████████████████████████████| 111 kB 50.8 MB/s 
     |████████████████████████████████| 141 kB 53.0 MB/s 
     |████████████████████████████████| 49 kB 6.0 MB/s 
     |████████████████████████████████| 1.1 MB 5.1 MB/s 


In [ ]:
from time import time
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import random
import os 
import cv2
import shutil
import random
import optuna  

import tensorflow as tf
import tensorflow.keras as K
import tensorflow_addons as tfa
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Lambda, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import backend as bk

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split


from tensorboard.plugins import projector

if tf.test.gpu_device_name()!='':
  physical_devices = tf.config.list_physical_devices('GPU')
  tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [ ]:
!cp /content/drive/MyDrive/Wipro/Data/Combined/combined_flickr_memotion_sen* .

In [ ]:
import pickle

with open('combined_flickr_memotion_sen_img_emb.pkl', 'rb') as f:
    image_embeddings = pickle.load(f)
with open('combined_flickr_memotion_sen_txt_emb.pkl', 'rb') as f:
    text_embeddings = pickle.load(f)

In [ ]:
len(image_embeddings.keys()), len(text_embeddings.keys())

(38610, 38610)

In [ ]:
len(list(text_embeddings.values())[0][0][0]), len(list(image_embeddings.values())[0][0])

(300, 4096)

# Triplet Loss

In [ ]:
def triplet_loss_image_to_text(y_true, y_pred):
    """
    Computes the unidirectional triplet loss with (anchor_image, positive_text, negative_text)
    
    Inputs:
    - y_true: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - y_pred: A numpy array of weights, of shape (4, M)
    
    Output:
    - triplet_loss:a scalar, so no shape

    """
    margin = 0.005
    
    #delete? embeddings = K.reshape(y_pred, (-1, 3, output_dim))
    anchor_image = y_pred[0]
    positive_text = y_pred[1]
    anchor_image_ignore = y_pred[2]
    negative_text = y_pred[3]
    
    positive_distance = K.backend.mean(K.backend.square(anchor_image - positive_text),axis=-1)
    negative_distance = K.backend.mean(K.backend.square(anchor_image - negative_text),axis=-1)
    
    triplet_loss = K.backend.mean(K.backend.maximum(0.0, positive_distance - negative_distance + margin))
    
    print("triplet_loss = ",triplet_loss)
    
    return triplet_loss

In [ ]:
def triplet_loss_text_to_image_prime(y_pred):
    """
    Computes the unidirectional triplet loss with (anchor_image, positive_text, negative_text)
    
    Inputs:
    - y_true: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - y_pred: A numpy array of weights, of shape (4, M)
    
    Output:
    - triplet_loss:a scalar, so no shape

    """
    margin = 0.005
    
    positive_image = y_pred[0]
    anchor_text = y_pred[1]
    negative_image = y_pred[2]
    anchor_text_ignore = y_pred[3]
    
    
    positive_distance = K.backend.mean(K.backend.square(anchor_text - positive_image),axis=-1)
    negative_distance = K.backend.mean(K.backend.square(anchor_text - negative_image),axis=-1)
    
    triplet_loss = K.backend.mean(K.backend.maximum(0.0, positive_distance - negative_distance + margin))
    
    return triplet_loss

In [ ]:
def bidirectional_triplet_loss_prime(y_pred):
    """
    Computes the bidirectional triplet loss with (anchor_image, positive_text, negative_text)

    Inputs:
    - y_true: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - y_pred: A numpy array of weights, of shape (4, M)
    
    Output:
    - triplet_loss:a scalar, so no shape
    """
    
    #hyperparameter to weight the ratio of direction of loss
    lambda_1 = 0.01
    
    bidirectional_triplet_loss = triplet_loss_image_to_text_prime(y_pred) + lambda_1 * triplet_loss_text_to_image_prime(y_pred)
    
    return bidirectional_triplet_loss

In [ ]:
def triplet_loss_image_to_text_prime(y_pred):
    """
    Computes the unidirectional triplet loss with (anchor_image, positive_text, negative_text)
    
    Inputs:
    - y_true: A numpy array containing input data, of shape (N, d_1, ..., d_k)
    - y_pred: A numpy array of weights, of shape (4, M)
    
    Output:
    - triplet_loss:a scalar, so no shape

    """
    margin = 0.1
    
    #delete? embeddings = K.reshape(y_pred, (-1, 3, output_dim))
    anchor_image = y_pred[0]
    positive_text = y_pred[1]
    anchor_image_ignore = y_pred[2]
    negative_text = y_pred[3]
    
    positive_distance = K.backend.mean(K.backend.square(anchor_image - positive_text),axis=-1)
    negative_distance = K.backend.mean(K.backend.square(anchor_image - negative_text),axis=-1)
    
    triplet_loss = K.backend.sum(K.backend.maximum(0.0, positive_distance - negative_distance + margin))
    
    print("triplet_loss = ",triplet_loss)
    
    return triplet_loss

In [ ]:
def identity_loss(y_true, y_pred):
    return K.backend.mean(y_pred - 0 * y_true)

# Network

In [ ]:
#image network
input_image = Input(shape=(4096,),name='input_image')
x = Dense(4096)(input_image)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
x = Dense(256)(x)
x = BatchNormalization()(x)
prediction_image = Lambda(lambda  x: bk.l2_normalize(x,axis=1))(x)

#text network
input_text = Input(shape=(300,),name='input_text')
y = Dense(4096)(input_text)
y = Activation('relu')(y)
y = Dropout(0.5)(y)
y = Dense(256)(y)
y = BatchNormalization()(y)
prediction_text = Lambda(lambda  x: bk.l2_normalize(x, axis=1))(y)


basic_model = Model(inputs=[input_image,input_text], outputs=[prediction_image,prediction_text])
#basic_model.compile(optimizer='rmsprop',
#              loss='categorical_crossentropy',
#              metrics=['accuracy'])
#print(basic_model.summary())

In [ ]:
#Shared model with multiple inputs
positive_image = Input(shape=(4096,))
positive_text = Input(shape=(300,))
negative_image = Input(shape=(4096,))
negative_text = Input(shape=(300,))
positive_basic_model = basic_model([positive_image,positive_text])
negative_basic_model = basic_model([negative_image,negative_text])

triplet_loss_image_to_text_PRIME = Lambda(triplet_loss_image_to_text_prime)([positive_basic_model[0], positive_basic_model[1],negative_basic_model[0],negative_basic_model[1]]) 

#merged_model = Model(inputs=[positive_image,positive_text,negative_image,negative_text], outputs=[positive_basic_model[0], positive_basic_model[1],negative_basic_model[0],negative_basic_model[1]])
merged_model = Model(inputs=[positive_image,positive_text,negative_image,negative_text], outputs=triplet_loss_image_to_text_PRIME)

ADAM = K.optimizers.Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#ADAM = K.optimizers.Adam()
merged_model.compile(optimizer=ADAM, loss=identity_loss, metrics=[])

#print(merged_model.summary())

triplet_loss =  Tensor("lambda_2/Sum:0", shape=(), dtype=float32)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning:

The `lr` argument is deprecated, use `learning_rate` instead.



# Triplet Selection

In [ ]:
#generate labels

def triplet_generator(image_embeddings, text_embeddings, margin, basic_model, batch_size, option):
    """
    Generate triplets based on the "option" value. 
    
    Inputs:
    - image_embeddings: A dictionary with values = embeddings of image, of shape (N, emb_image)
    - text_embeddings: A dictionary with values =  embeddings of text, of shape (N, emb_text)
    - margin: A scalar specifying margin
    - basic_model: A numpy array of weights, of shape (4, M)?
    - batch_size: A scalar specifying mini batch size
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives
    
    Output:
    - image_text_pair_keys: A list of randomly chosen image_text_pair_keys of size k. 
        This might be useful for post processing later. 
    - quadrapule:(positive_image_list, positive_text_list, negative_image_list, negative_txt_list)

    """
    #k hardest examples
    K=1
    
    #Create a list of randomly chosen image_text_pair_keys of size k
    #Example of an element in image_text_pair_keys = 31034578.png
    image_text_pair_keys = random.choices(list(text_embeddings.keys()),k=batch_size)
    
    #create a list of corresponding text_embeddings for image_text_pair
    text_embedding_of_image_text_pairs=list()
    image_embedding_of_image_text_pairs=list()
    for image_text_pair_key in image_text_pair_keys:
        text_embedding_of_image_text_pairs.append(random.choice(text_embeddings[image_text_pair_key])) #do you need random.choice here?
        image_embedding_of_image_text_pairs.append(random.choice(image_embeddings[image_text_pair_key])) #do you need random.choice here?
    
    #can we eliminate loops here?
    #positive_image_text_pair_embeddings is a list of size(2,batch_size, ????)
    positive_image_text_pair_embedding = [[np.array(image_embedding_of_image_text_pairs[idx]).reshape(-1,4096),np.array(text_embedding_of_image_text_pairs[idx]).reshape(-1,300)] for idx in range(len(image_text_pair_keys))]
    
    #image_embedding_minibatch = np.array(image_embedding_of_image_text_pairs)
    #text_embedding_minibatch = np.array(text_embedding_of_image_text_pairs)
             
    predictions = list()
    #positives_image = list()
    #positives_text = list()
                                            
    for inst in positive_image_text_pair_embedding:
        #print(inst[0].shape)
        #print(inst[1].shape)
        #print(type(inst))
        predictions.append(basic_model.predict(inst))
    
    #image_joint_embedding is numpy array of size(batch_size, N )                                    
    image_joint_embedding = np.vstack([prediction[0] for prediction in predictions])
    #text_joint_embedding is numpy array of size(batch_size, N)
    text_joint_embedding = np.vstack([prediction[1] for prediction in predictions])
    
    #print(image_joint_embedding.shape)
    #print(text_joint_embedding.shape)
    
    if option in [0,1,5,6]:
        generate_mini_batch_of_triplets = generate_mini_batch_of_triplets_K_hardest_negatives
    elif option == 2 or option == 7:
        generate_mini_batch_of_triplets = generate_mini_batch_of_triplets_ALL_hard_negatives
    elif option == 3 or option == 8:
        generate_mini_batch_of_triplets = generate_mini_batch_of_triplets_ALL_semihard_negatives
    elif option == 4 or option == 9:
        generate_mini_batch_of_triplets = generate_mini_batch_of_triplets_ALL_hard_and_semihard_negatives
        
    positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch = generate_mini_batch_of_triplets(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option,batch_size,margin)
    
    return image_text_pair_keys, positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

In [ ]:
def generate_mini_batch_of_triplets_ALL_hard_negatives(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option, batch_size, margin):
    """
    Generates triplets of ALL hardest negatives.
    
    Inputs:
    - positive_image_text_pair_embedding: A list of size (2,) [image_embedding, text_embedding].
        image_embeddding is numpy array of size (batch_size,emb_image)
        text_embeddding is numpy array of size (batch_size,emb_text)
    - image_joint_embedding: the joint embedding from the output of base_model from image_embedding input of size (batch_size,emb_joint)
    - text_joint_embedding: the joint embedding from the outout of base_model from text_embedding input of size (batch_size,emb_joint)
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives
    - batch_size: A scalar representing mini batch size
    - margin: A scalar or hyperparameter as defined in margin-based ranking loss
    
    Output:
    - quadrapule:(positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch)
        Total number of triplets generates = cannot say for sure, but it will be between [batch_size,batch*size * 2]
    """
    if option in [0,1,2,3,4]:
        set_metric = 'euclidean'
    else:
        set_metric = 'cosine'
    
    K=1
        
    # Compute similarity score among minibatch
    nearest_neighbour_model = NearestNeighbors(n_neighbors=K+1,metric = set_metric) #batch_size
    nearest_neighbors = nearest_neighbour_model.fit(text_joint_embedding)
    distances, indices = nearest_neighbors.kneighbors(image_joint_embedding)
    
    #print(max(distance))
    #print(min(distance))
    
    positive_image_embedding_minibatch = np.empty(shape=[0, 4096])
    positive_text_embedding_minibatch = np.empty(shape=[0, 300])
    negative_image_embedding_minibatch = np.empty(shape=[0, 4096])
    negative_text_embedding_minibatch = np.empty(shape=[0, 300])
    
    for i in range(batch_size):
        #positive_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        #positive_text_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][1]                      
        #negative_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        
        #start from hardest negative and check if is hard negative or not?
        j = 0
        while (j < K+1) and (j != indices[i,j]):#batch_size
            np.append(positive_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
            np.append(positive_text_embedding_minibatch, positive_image_text_pair_embedding[i][1],axis=0)                   
            np.append(negative_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
            np.append(negative_text_embedding_minibatch, positive_image_text_pair_embedding[indices[i,j]][1],axis=0)
            j = j+1
    return positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

In [ ]:
def generate_mini_batch_of_triplets_ALL_hard_and_semihard_negatives(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option, batch_size, margin):
    """
    Generates triplets of ALL hard and semihard negatives.
    
    Inputs:
    - positive_image_text_pair_embedding: A list of size (2,) [image_embedding, text_embedding].
        image_embeddding is numpy array of size (batch_size,emb_image)
        text_embeddding is numpy array of size (batch_size,emb_text)
    - image_joint_embedding: the joint embedding from the output of base_model from image_embedding input of size (batch_size,emb_joint)
    - text_joint_embedding: the joint embedding from the outout of base_model from text_embedding input of size (batch_size,emb_joint)
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives
    - batch_size: A scalar representing mini batch size
    - margin: A scalar or hyperparameter as defined in margin-based ranking loss
    
    Output:
    - quadrapule:(positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch)
        Total number of triplets generates = cannot say for sure, but it will be between [batch_size,batch*size * 2]
    """
    if option in [0,1,2,3,4]:
        set_metric = 'euclidean'
    else:
        set_metric = 'cosine'
        
    K = 1
        
    # Compute similarity score among minibatch
    nearest_neighbour_model = NearestNeighbors(n_neighbors=K+1,metric = set_metric)#batch_size
    nearest_neighbors = nearest_neighbour_model.fit(text_joint_embedding)
    distances, indices = nearest_neighbors.kneighbors(image_joint_embedding)
    
    positive_to_anchor_distance = np.zeros((batch_size,))
    
    for idx in range(batch_size):
        for j in range(K+1):#batch_size
            if idx == indices[idx,j]:
                positive_to_anchor_distance[idx] = distances[idx,j]
                
    positive_image_embedding_minibatch = np.empty(shape=[0, 4096])
    positive_text_embedding_minibatch = np.empty(shape=[0, 300])
    negative_image_embedding_minibatch = np.empty(shape=[0, 4096])
    negative_text_embedding_minibatch = np.empty(shape=[0, 300])
    
    #positive_image_embedding_minibatch = np.zeros((batch_size*K,4096))
    #positive_text_embedding_minibatch = np.zeros((batch_size*K,300))
    #negative_image_embedding_minibatch = np.zeros((batch_size*K,4096))
    #negative_text_embedding_minibatch = np.zeros((batch_size*K,300))
    
    for i in range(batch_size):
        #positive_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        #positive_text_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][1]                      
        #negative_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        
        #start from hardest negative and check if is hard negative or not?
        j = 0
        while (j < K+1) and (distances[i,j] <= positive_to_anchor_distance[i]+margin):
            np.append(positive_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
            np.append(positive_text_embedding_minibatch, positive_image_text_pair_embedding[i][1],axis=0)                     
            np.append(negative_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
            np.append(negative_text_embedding_minibatch, positive_image_text_pair_embedding[indices[i,j]][1],axis=0)
            j = j+1
    return positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

In [ ]:
def generate_mini_batch_of_triplets_ALL_semihard_negatives(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option, batch_size, margin):
    """
    Generates triplets of ALL hardest negatives.
    
    Inputs:
    - positive_image_text_pair_embedding: A list of size (2,) [image_embedding, text_embedding].
        image_embeddding is numpy array of size (batch_size,emb_image)
        text_embeddding is numpy array of size (batch_size,emb_text)
    - image_joint_embedding: the joint embedding from the output of base_model from image_embedding input of size (batch_size,emb_joint)
    - text_joint_embedding: the joint embedding from the outout of base_model from text_embedding input of size (batch_size,emb_joint)
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives
    - batch_size: A scalar representing mini batch size
    - margin: A scalar or hyperparameter as defined in margin-based ranking loss
    
    Output:
    - quadrapule:(positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch)
        Total number of triplets generated = cannot say for sure, but it will be between [batch_size,batch*size * 2]
    """
    if option in [0,1,2,3,4]:
        set_metric = 'euclidean'
    else:
        set_metric = 'cosine'
        
    K = 1
        
    # Compute similarity score among minibatch
    nearest_neighbour_model = NearestNeighbors(n_neighbors=K+1,metric = set_metric)#batch_size
    nearest_neighbors = nearest_neighbour_model.fit(text_joint_embedding)
    distances, indices = nearest_neighbors.kneighbors(image_joint_embedding)
    
    positive_to_anchor_distance = np.zeros((batch_size,))
    
    for idx in range(batch_size):
        for j in range(K+1): #batch_size
            if idx == indices[idx,j]:
                positive_to_anchor_distance[idx] = distances[idx,j]
    
    positive_image_embedding_minibatch = np.empty(shape=[0, 4096])
    positive_text_embedding_minibatch = np.empty(shape=[0, 300])
    negative_image_embedding_minibatch = np.empty(shape=[0, 4096])
    negative_text_embedding_minibatch = np.empty(shape=[0, 300])
    
    for i in range(batch_size):
        #positive_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        #positive_text_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][1]                      
        #negative_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        
        #start from hardest negative and check if is hard negative or not?
        j = 0
        while (j < K+1) and (distances[i,j] <= positive_to_anchor_distance[i]+margin):#batch_size
            if distances[i,j] > positive_to_anchor_distance[i]:
                np.append(positive_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
                np.append(positive_text_embedding_minibatch, positive_image_text_pair_embedding[i][1],axis=0)                     
                np.append(negative_image_embedding_minibatch, positive_image_text_pair_embedding[i][0],axis=0)
                np.append(negative_text_embedding_minibatch, positive_image_text_pair_embedding[indices[i,j]][1],axis=0)
            j = j+1
    return positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

In [ ]:
def generate_mini_batch_of_triplets_K_hardest_negatives(positive_image_text_pair_embedding,image_joint_embedding,text_joint_embedding,option, batch_size, margin):
    """
    Generates triplets of K hardest negatives. 
    
    Inputs:
    - positive_image_text_pair_embedding: A list of size (2,) [image_embedding, text_embedding].
        image_embeddding is numpy array of size (batch_size,emb_image)
        text_embeddding is numpy array of size (batch_size,emb_text)
    - image_joint_embedding: the joint embedding from the output of base_model from image_embedding input of size (batch_size,emb_joint)
    - text_joint_embedding: the joint embedding from the outout of base_model from text_embedding input of size (batch_size,emb_joint)
    - option: a choice from 0 to 9 listed below
        0 = Euclidean Distance + K=1 hardest negative
        1 = Euclidean Distance + K=10 hardest negatives
        2 = Euclidean Distance + ALL hard only negatives
        3 = Euclidean Distance + ALL semi-hard only negatives
        4 = Euclidean Distance + ALL hard and semi-hard negatives
        5 = Cosine Distance + K=1 hardest negative
        6 = Cosine Distance + K=10 hardest negatives
        7 = Cosine Distance + ALL hard only negatives
        8 = Cosine Distance + ALL semi-hard only negatives
        9 = Cosine Distance + ALL hard and semi-hard negatives
    - batch_size: A scalar representing mini batch size
    - margin: A scalar or hyperparameter as defined in margin-based ranking loss
    
    Output:
    - quadrapule:(positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch)
    Total number of triplets generated = batch_size * K    
    """
    if option in [0,1,2,3,4]:
        set_metric = 'euclidean'
    else:
        set_metric = 'cosine'
        
    if option == 0:
        K=1
    elif option == 1:
        K=10
        
    # Compute similarity score among minibatch
    nearest_neighbour_model = NearestNeighbors(n_neighbors=K+1, metric = set_metric)
    nearest_neighbors = nearest_neighbour_model.fit(text_joint_embedding)
    distances, indices = nearest_neighbors.kneighbors(image_joint_embedding)
    
    #print("Max_distance = ",np.max(distances))
    #print("Min_distance = ",np.min(distances))
    
    positive_image_embedding_minibatch = np.zeros((batch_size*K,4096))
    positive_text_embedding_minibatch = np.zeros((batch_size*K,300))
    negative_image_embedding_minibatch = np.zeros((batch_size*K,4096))
    negative_text_embedding_minibatch = np.zeros((batch_size*K,300))
    
    for i in range(batch_size):
        #positive_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        #positive_text_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][1]                      
        #negative_image_embedding_minibatch[i,:] = positive_image_text_pair_embeddings[i%K][0]
        
        #STATE is set to 1, when the positive text is found in the nearest K neighbours. 
        # When STATE is set to 1, we just skip over that neighbouring point. 
        STATE = 0
        
        #print("#"*20,i,"#"*20)
        #print("distance =",distances[i])
        #print("indices =",indices[i])
        #print("Min_distance = ",np.min(distances[i]))
    
        for j in range(0,K):
            positive_image_embedding_minibatch[i+j,:] = positive_image_text_pair_embedding[i][0]
            positive_text_embedding_minibatch[i+j,:] = positive_image_text_pair_embedding[i][1]                      
            negative_image_embedding_minibatch[i+j,:] = positive_image_text_pair_embedding[i][0]
        
            if i == indices[i,j]:
                STATE = 1
            #is this correct? Does this not mean that nearest neighbour is same as positive text?
            #shouldn't this be text_embedding_minibatch[indices[i,1]] instead???
                #negative_text_embedding_minibatch[i+j,:] = positive_image_text_pair_embeddings[indices[i,j+state]][1] 
            #else:
            #is this correct? Does this not mean that nearest neighbour is NOT same as positive text?
            #shouldn't this be text_embedding_minibatch[indices[i,0]] instead???
            #hard negative example
                #negative_text_embedding_minibatch[i+j,:] = positive_image_text_pair_embeddings[indices[i,j+state]][1]
            negative_text_embedding_minibatch[i+j,:] = positive_image_text_pair_embedding[indices[i,j+STATE]][1]
    

    return positive_image_embedding_minibatch, positive_text_embedding_minibatch, negative_image_embedding_minibatch, negative_text_embedding_minibatch

#Split Test and Train


In [ ]:
from random import seed, sample

seed(42)

test_validation_keys = sample(image_embeddings.keys(),k=2000)

validation_keys = test_validation_keys[:1000]
test_keys = test_validation_keys[1000:]

test_keys = set(test_keys)
validation_keys = set(validation_keys)

#print(list(test_embeddings.keys())[:10])
#print(choices)

test_image = dict()
test_text = dict()

validation_image = dict()
validation_text = dict()

train_image = dict()
train_text = dict()

for item in image_embeddings.keys():
    if item in test_keys:
        test_image[item] = image_embeddings[item]
        test_text[item] = text_embeddings[item]
    elif item in validation_keys:
        validation_image[item] = image_embeddings[item]
        validation_text[item] = text_embeddings[item]
    else:
        train_image[item] = image_embeddings[item]
        train_text[item] = text_embeddings[item]

In [ ]:
len(test_image.keys()), len(test_text.keys())

(1000, 1000)

# Evaluation Setup

In [ ]:
import itertools
def image_to_text_recall_at_k(image_embeddings, image_labels, text_embeddings, text_labels, k=[1, 2, 5, 10], metric='euclidean'):
    """
    args:
        embeddings : mini_batch embedding array 
                     size = [nof_samples x feature_length]
        labels     : labels corresponding to embeddings
                     size: [nof_samples,]
        k          : recall/precision@k
        metric     : distance metric to evaluate 
                     available choices: cosine, euclidean, l1, l2, manhattan, mahalanobis,hamming
        
    Measure the recall score at k:
    Each test image (query) first retrieves K nearest neighbors from the test set
    and receives score 1 if an image of the same class is retrieved among the K
    nearest neighbors and 0 otherwise. Recall@K averages this score over all the images.
    
    Measure precision at k:
    Each test image (query) first retrieves K nearest neighbors from the test set
    precision_at_k is average number of correct labels in k neighborhood
    for each image in test set
    """   
#    indices = np.transpose(np.argsort(cdist(embeddings, embeddings, metric='euclidean'), axis=0)[:n,:])
    
    # Find the nearest k[-1]+1 neighbors of each image embedding (the neighbors are text embeddings)
    nbrs = NearestNeighbors(n_neighbors=k[-1], algorithm='auto', metric= metric, n_jobs=-1).fit(text_embeddings)
    distances, indices = nbrs.kneighbors(image_embeddings) 
    
    print(indices[:25])
    
    #print(indices)
    
    #print(distances)
    
    neighbors_array = [text_labels[i] for i in list(itertools.chain(*indices))]
    neighbors_array = np.reshape(np.asarray(neighbors_array, dtype=np.int32), indices.shape)
    
    #for i in neighbors_array:
        #print(i)
        
    #search_array = [image_labels[i] for i in range(image_labels.shape[1])]
    #query_array = np.repeat(np.expand_dims(neighbors_array[:, 0], 1), k[-1]+1, axis=1)
    sparse_array = np.equal(neighbors_array, image_labels).astype(np.float32)
    
    #print(sparse_array)
    #for i in query_array:
        #print(i)

    recall_at_k = np.zeros((len(k)), )
    prec_at_k = np.zeros((len(k)), )
    
    for i, n in enumerate(k):
        recall_at_k[i] = np.mean(np.sum(sparse_array[:, :n], axis=1).astype(np.bool).astype(np.float32), axis=0)
        prec_at_k[i] = np.mean(np.mean(sparse_array[:, :n], axis=1), axis=0)

    return prec_at_k, recall_at_k

In [ ]:
def evaluate_image_to_text(test_images, test_texts, basic_model):
    #Convert image names into ints
    label_dict = dict()
    
    count=0
    array_count = 0
    image_embeddings = np.empty([len(test_image.keys()),256])
    image_labels = np.empty([len(test_image.keys()),1])
    
    num_of_texts = sum([len(emb)for emb in test_text.values()])
    text_embeddings = np.empty([num_of_texts,256])
    text_labels = np.empty([num_of_texts,1])
    
    
    for name in tqdm(test_images.keys()):
        if name not in label_dict.keys():
            label_dict[name] = count
            count += 1
        image_embeddings[array_count,:] = basic_model.predict([test_images[name],test_texts[name][0]])[0]
        image_labels[array_count]=label_dict[name]
        array_count += 1
        
    array_count = 0
    for name in tqdm(test_texts.keys()):
        for embedding in test_texts[name]:
            #print(embedding)
            prediction = basic_model.predict([test_images[name],embedding])
            #print(prediction)
            text_embeddings[array_count,:] = prediction[1]
            text_labels[array_count] = label_dict[name]
            array_count +=1
            
    for i,key in enumerate(test_images.keys()):
        print("########",key,"########",image_labels[i])
            
    p,r = image_to_text_recall_at_k(image_embeddings, image_labels, text_embeddings, text_labels, k=[1, 2, 5, 10], metric='euclidean')
    #print(image_embeddings.shape, image_labels[:10,], text_embeddings[:10,], text_labels[:10,])
    
    return p,r
    

In [ ]:
def evaluate_text_to_image(test_images, test_texts, basic_model):
    #Convert image names into ints
    label_dict = dict()
    
    count=0
    array_count = 0
    image_embeddings = np.empty([len(test_image.keys()),256])
    image_labels = np.empty([len(test_image.keys()),1])
    
    num_of_texts = sum([len(emb)for emb in test_text.values()])
    text_embeddings = np.empty([num_of_texts,256])
    text_labels = np.empty([num_of_texts,1])
    
    
    for name in tqdm(test_images.keys()):
        if name not in label_dict.keys():
            label_dict[name] = count
            count += 1
        image_embeddings[array_count,:] = basic_model.predict([test_images[name],test_texts[name][0]])[0]
        image_labels[array_count]=label_dict[name]
        array_count += 1
        
    array_count = 0
    for name in tqdm(test_texts.keys()):
        for embedding in test_texts[name]:
            #print(embedding)
            prediction = basic_model.predict([test_images[name],embedding])
            #print(prediction)
            text_embeddings[array_count,:] = prediction[1]
            text_labels[array_count] = label_dict[name]
            array_count +=1
            
    for i,key in enumerate(test_images.keys()):
        print("########",key,"########",image_labels[i])
            
    p,r = image_to_text_recall_at_k(text_embeddings, text_labels, image_embeddings, image_labels, k=[1, 2, 5, 10], metric='euclidean')
    #print(image_embeddings.shape, image_labels[:10,], text_embeddings[:10,], text_labels[:10,])
    
    return p,r

In [ ]:
np.set_printoptions(threshold=np.inf)
p,r = evaluate_image_to_text(test_image, test_text, basic_model)
print(p,r)

#p,r = evaluate_image_to_text(validation_image, validation_text, basic_model)
#print(p,r)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1000 [00:00<?, ?it/s]

######## 102998070.jpg ######## [0.]
######## 1061887159.jpg ######## [1.]
######## 1112334131.jpg ######## [2.]
######## 1129704496.jpg ######## [3.]
######## 1160034462.jpg ######## [4.]
######## 1160441615.jpg ######## [5.]
######## 1167669558.jpg ######## [6.]
######## 116976749.jpg ######## [7.]
######## 1204996216.jpg ######## [8.]
######## 1295719054.jpg ######## [9.]
######## 1306755031.jpg ######## [10.]
######## 1325756278.jpg ######## [11.]
######## 1332815795.jpg ######## [12.]
######## 1335877186.jpg ######## [13.]
######## 1338282180.jpg ######## [14.]
######## 1361790462.jpg ######## [15.]
######## 136679463.jpg ######## [16.]
######## 13691837.jpg ######## [17.]
######## 1370773415.jpg ######## [18.]
######## 1395173766.jpg ######## [19.]
######## 1403414927.jpg ######## [20.]
######## 1409731198.jpg ######## [21.]
######## 1415310632.jpg ######## [22.]
######## 1432506301.jpg ######## [23.]
######## 1460352062.jpg ######## [24.]
######## 146906547.jpg ######## [25.]
##

# Training

In [ ]:
#Initialize Training Parameters
batch_size = 64
precision = list()
recall = list()
losses = list()
epochs = list()
precision_validation = list()
recall_validation = list()

In [ ]:
for i in tqdm(range(2000)):
    _,positive_images_minibatch, positive_texts_minibatch, negative_images_minibatch, negative_texts_minibatch = triplet_generator(
        train_image,train_text,0.1,basic_model,batch_size = 64, option=0)
    minibatch_shape = positive_images_minibatch.shape
    
    loss = merged_model.train_on_batch(
        [positive_images_minibatch, positive_texts_minibatch, negative_images_minibatch, negative_texts_minibatch],
        np.ones(minibatch_shape[0]))
    losses.append(loss)
    if len(epochs)>0:
        epochs.append(epochs[-1] + 1)
    else:
        epochs.append(1)
    
    if i % 500 == 0:
        p,r = evaluate_image_to_text(test_image, test_text, basic_model)
        p_validation,r_validation = evaluate_image_to_text(validation_image, validation_text, basic_model)
        print('Precision')
        print(p)
        print('Recall')
        print(r)
        print('Validation_Precision')
        print(p_validation)
        print('Validation_Recall')
        print(r_validation)
        print('Loss')
        print(loss)
        precision.append(p)
        recall.append(r)
        precision_validation.append(p_validation)
        recall_validation.append(r_validation)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/2000 [00:00<?, ?it/s]

triplet_loss =  Tensor("model_1/lambda_2/Sum:0", shape=(), dtype=float32)
triplet_loss =  Tensor("model_1/lambda_2/Sum:0", shape=(), dtype=float32)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1000 [00:00<?, ?it/s]

...

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1000 [00:00<?, ?it/s]

######## 100716317.jpg ######## [0.]
######## 1010031975.jpg ######## [1.]
######## 1021293940.jpg ######## [2.]
######## 1032122270.jpg ######## [3.]
######## 10459869.jpg ######## [4.]
######## 1056249424.jpg ######## [5.]
######## 1057210460.jpg ######## [6.]
######## 107532584.jpg ######## [7.]
######## 1082110795.jpg ######## [8.]
######## 1118557877.jpg ######## [9.]
######## 1123486015.jpg ######## [10.]
######## 1138784872.jpg ######## [11.]
######## 114949897.jpg ######## [12.]
######## 1163297375.jpg ######## [13.]
######## 1201218421.jpg ######## [14.]
######## 1212131969.jpg ######## [15.]
######## 122427967.jpg ######## [16.]
######## 1250845715.jpg ######## [17.]
######## 126762293.jpg ######## [18.]
######## 1275693139.jpg ######## [19.]
######## 1290894194.jpg ######## [20.]
######## 129860826.jpg ######## [21.]
######## 1305564994.jpg ######## [22.]
######## 131632409.jpg ######## [23.]
######## 134867998.jpg ######## [24.]
######## 1377428277.jpg ######## [25.]
######

In [ ]:
import pandas as pd
import altair as alt
#May make notebook large
alt.data_transformers.enable('default', max_rows=None)

df = pd.DataFrame({'Epoch':epochs,'Loss':losses})

df['Rolling'] = df['Loss'].rolling(250, center=True).mean()

orig = alt.Chart(df).mark_line().encode(
    x='Epoch',
    y='Loss',
).interactive()

mean = alt.Chart(df).mark_line(color='red').encode(
    x='Epoch',
    y='Rolling',
).interactive()

orig + mean

alt.LayerChart(...)

In [ ]:
print(precision)

[array([0.002 , 0.0015, 0.0016, 0.0015]), array([0.071     , 0.0665    , 0.0596    , 0.05050001]), array([0.081 , 0.0805, 0.0648, 0.0577]), array([0.055 , 0.06  , 0.0548, 0.0497])]


In [ ]:
print(recall)

[array([0.002, 0.003, 0.008, 0.014]), array([0.071     , 0.118     , 0.219     , 0.30899999]), array([0.081     , 0.13699999, 0.21699999, 0.32699999]), array([0.055     , 0.109     , 0.20100001, 0.32100001])]


#Save Outputs

In [ ]:
!mkdir pickles

with open('pickles/stanford_scratch_combined_memotion_sentences_stats.pkl', 'wb') as f:
  pickle.dump([precision_validation, recall_validation, precision, recall, losses, epochs],f)
  
#!gsutil cp pickles/performance.pkl gs://rnd_datasets/flickr30k/pickles/performance_k1_vgg19_lrDOT0002_marginDOT1_final.pkl

In [ ]:
from tensorflow.keras.models import save_model
!mkdir models

save_model(
    basic_model,
    'models/stanford_scratch_combined_memotion_sentences_basic_model.hdf5',
    overwrite=True,
    include_optimizer=True
)

from tensorflow.keras.models import save_model

save_model(
    merged_model,
    'models/stanford_scratch_combined_memotion_sentences_merged_model.hdf5',
    overwrite=True,
    include_optimizer=True
)

"""
!gsutil rsync -r models/ gs://rnd_datasets/flickr30k/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/
"""

'\n!gsutil rsync -r models/ gs://rnd_datasets/flickr30k/pickles/models_k1_vgg19_lrDOT0002_marginDOT1_final/\n'

In [ ]:
"""
!mkdir pickles
!gsutil cp gs://rnd_datasets/flickr30k/pickles/performance_k1_vgg19_lrDOT0002_marginDOT1.pkl
import pickle
"""
with open('pickles/stanford_scratch_combined_memotion_sentences_stats.pkl', 'rb') as f:
    validation_precision,validation_recall,precision, recall, losses, epochs = pickle.load(f)

In [ ]:
!cp -r pickles models /content/drive/MyDrive/Wipro/Implementation/

In [ ]:
print(precision)

[array([0.002 , 0.0015, 0.0016, 0.0015]), array([0.071     , 0.0665    , 0.0596    , 0.05050001]), array([0.081 , 0.0805, 0.0648, 0.0577]), array([0.055 , 0.06  , 0.0548, 0.0497])]


In [ ]:
validation_recall_1 = list()
validation_recall_5 = list()
validation_recall_10 = list()

test_recall_1 = list()
test_recall_5 = list()
test_recall_10 = list()

print('#'*40)
for i,r_v in enumerate(recall_validation):
    print("The validation recall when epoch/500=",i,"="*10, r_v)
    validation_recall_1.append(r_v[0])
    validation_recall_5.append(r_v[2])
    validation_recall_10.append(r_v[3])
      
print('#'*40)
for i,r in enumerate(recall):
    print("The test recall when epoch/500=",i,"="*10,r)
    test_recall_1.append(r[0])
    test_recall_5.append(r[2])
    test_recall_10.append(r[3])
    
print(validation_recall_1)
print(test_recall_1)

########################################
The validation recall when epoch/500= 0 ========== [0.001 0.001 0.008 0.016]
The validation recall when epoch/500= 1 ========== [0.08       0.124      0.226      0.32600001]
The validation recall when epoch/500= 2 ========== [0.089      0.14399999 0.245      0.34599999]
The validation recall when epoch/500= 3 ========== [0.082      0.125      0.222      0.31299999]
########################################
The test recall when epoch/500= 0 ========== [0.002 0.003 0.008 0.014]
The test recall when epoch/500= 1 ========== [0.071      0.118      0.219      0.30899999]
The test recall when epoch/500= 2 ========== [0.081      0.13699999 0.21699999 0.32699999]
The test recall when epoch/500= 3 ========== [0.055      0.109      0.20100001 0.32100001]
[0.0010000000474974513, 0.07999999821186066, 0.08900000154972076, 0.0820000022649765]
[0.0020000000949949026, 0.07100000232458115, 0.08100000023841858, 0.054999999701976776]


In [ ]:
np.set_printoptions(threshold=np.inf)
p,r = evaluate_image_to_text(test_image, test_text, basic_model)
print(p,r)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1000 [00:00<?, ?it/s]

######## 102998070.jpg ######## [0.]
######## 1061887159.jpg ######## [1.]
######## 1112334131.jpg ######## [2.]
######## 1129704496.jpg ######## [3.]
######## 1160034462.jpg ######## [4.]
######## 1160441615.jpg ######## [5.]
######## 1167669558.jpg ######## [6.]
######## 116976749.jpg ######## [7.]
######## 1204996216.jpg ######## [8.]
######## 1295719054.jpg ######## [9.]
######## 1306755031.jpg ######## [10.]
######## 1325756278.jpg ######## [11.]
######## 1332815795.jpg ######## [12.]
######## 1335877186.jpg ######## [13.]
######## 1338282180.jpg ######## [14.]
######## 1361790462.jpg ######## [15.]
######## 136679463.jpg ######## [16.]
######## 13691837.jpg ######## [17.]
######## 1370773415.jpg ######## [18.]
######## 1395173766.jpg ######## [19.]
######## 1403414927.jpg ######## [20.]
######## 1409731198.jpg ######## [21.]
######## 1415310632.jpg ######## [22.]
######## 1432506301.jpg ######## [23.]
######## 1460352062.jpg ######## [24.]
######## 146906547.jpg ######## [25.]
##

In [ ]:
np.set_printoptions(threshold=np.inf)
p,r = evaluate_text_to_image(test_image, test_text, basic_model)
print(p,r)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



  0%|          | 0/1000 [00:00<?, ?it/s]

######## 102998070.jpg ######## [0.]
######## 1061887159.jpg ######## [1.]
######## 1112334131.jpg ######## [2.]
######## 1129704496.jpg ######## [3.]
######## 1160034462.jpg ######## [4.]
######## 1160441615.jpg ######## [5.]
######## 1167669558.jpg ######## [6.]
######## 116976749.jpg ######## [7.]
######## 1204996216.jpg ######## [8.]
######## 1295719054.jpg ######## [9.]
######## 1306755031.jpg ######## [10.]
######## 1325756278.jpg ######## [11.]
######## 1332815795.jpg ######## [12.]
######## 1335877186.jpg ######## [13.]
######## 1338282180.jpg ######## [14.]
######## 1361790462.jpg ######## [15.]
######## 136679463.jpg ######## [16.]
######## 13691837.jpg ######## [17.]
######## 1370773415.jpg ######## [18.]
######## 1395173766.jpg ######## [19.]
######## 1403414927.jpg ######## [20.]
######## 1409731198.jpg ######## [21.]
######## 1415310632.jpg ######## [22.]
######## 1432506301.jpg ######## [23.]
######## 1460352062.jpg ######## [24.]
######## 146906547.jpg ######## [25.]
##